# Project

## Download Data

In [1]:
import urllib
import json
import pandas as pd
from bd_addons.interpro_data import *
from bd_addons.HmmPy import *
from bd_addons.metrics import *
from Bio import SearchIO, SeqIO

url = "https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/pf03060?format=json"

In [2]:
metadata, entries, gt = get_data(url, 1)


# 8 - Evaluate Ability of Matching sequences
Once we made the PSI-BLAST e HMM-searches we will have some files where all the sequences are... we should:

- Find a format that we can parse (there should be options in the documentations of thevarious methods);
- Decide a threshold for the E-value and take all the hits that are below that E value;
- All the remaining sequences (call them OUT_SEQs)are the ones that, according to our model, belong to the Domain family.
- Make a Confusion Matrix comparing OUT_SEQs with the ground truth.
- compute accuracy, sensitivity, specificty etc

## HMM Searches Performance

In [3]:
parsed_tblouts, parsed_domtblouts = parse_hmms()

In [4]:
parsed_psiblast = parse_psiblast()

## PSSM Model Performance (C1_denoised1)

## PSSM Model Performance (C1)

In [6]:
# hmmsearch_out_C_1_denoised1_swissprot_tblout
parsed_tblouts_1 = parsed_tblouts[0]
metrics_list = metrics_sequences(parsed_tblouts_1, gt)
metrics_list

[0.9962019391033597,
 0.010623556581986143,
 1.0,
 0.9962017842038908,
 0.9981008921019454,
 0.10287471031096798,
 0.02102376599634369]

In [8]:
# out_psiblast_C_1_swissprot_1iterations
parsed_psiblast_3 = parsed_psiblast[3]
metrics_list = metrics_sequences(parsed_psiblast_3, gt)
metrics_list

[0.9998173668196293,
 0.1774193548387097,
 0.9565217391304348,
 0.9998191325811376,
 0.9781704358557862,
 0.41191222896147284,
 0.29931972789115646]

In [7]:
# out_psiblast_C_1_denoised1_swissprot_1iterations
parsed_psiblast_1 = parsed_psiblast[0]
metrics_list = metrics_sequences(parsed_psiblast_1, gt)
metrics_list

[0.9997588532764038,
 0.14012738853503184,
 0.9565217391304348,
 0.9997606166515057,
 0.9781411778909703,
 0.3660596305595816,
 0.24444444444444444]

In [9]:
# out_psiblast_M_3_uniref90_1iterations
parsed_psiblast_boh = parsed_psiblast[7]
metrics_list = metrics_sequences(parsed_psiblast_boh, gt)
metrics_list

[0.9998226862326498,
 0.18181818181818182,
 0.9565217391304348,
 0.9998244522111042,
 0.9781730956707695,
 0.41698852454366625,
 0.3055555555555556]

In [10]:
# out_psiblast_O_3_uniref90_1iterations
parsed_psiblast_bohh = parsed_psiblast[10]
metrics_list = metrics_sequences(parsed_psiblast_bohh, gt)
metrics_list

[0.9996028171611356,
 0.08979591836734693,
 0.9565217391304348,
 0.9996045741724873,
 0.978063156651461,
 0.2930094611063365,
 0.16417910447761194]

# 9 - Evualuate the ability of matching domain positions

*TODO* Scrivere tabella riassuntiva dei risultati delle metriche e selezione del modello